<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Manipulate-Hamiltonian-into-blocks" data-toc-modified-id="Manipulate-Hamiltonian-into-blocks-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Manipulate Hamiltonian into blocks</a></span></li><li><span><a href="#Feature-computation" data-toc-modified-id="Feature-computation-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Feature computation</a></span></li><li><span><a href="#Feature-Preprocessing" data-toc-modified-id="Feature-Preprocessing-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Feature Preprocessing</a></span></li><li><span><a href="#Dataset" data-toc-modified-id="Dataset-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Dataset</a></span></li><li><span><a href="#DataLoader" data-toc-modified-id="DataLoader-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>DataLoader</a></span></li><li><span><a href="#Model" data-toc-modified-id="Model-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Model</a></span></li><li><span><a href="#Training" data-toc-modified-id="Training-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Training</a></span></li><li><span><a href="#Evaluation" data-toc-modified-id="Evaluation-8"><span class="toc-item-num">8&nbsp;&nbsp;</span>Evaluation</a></span></li><li><span><a href="#Tests" data-toc-modified-id="Tests-9"><span class="toc-item-num">9&nbsp;&nbsp;</span>Tests</a></span><ul class="toc-item"><li><span><a href="#set-up-wigner-d-rotations-matrices" data-toc-modified-id="set-up-wigner-d-rotations-matrices-9.1"><span class="toc-item-num">9.1&nbsp;&nbsp;</span>set up wigner-d rotations matrices</a></span></li><li><span><a href="#block-wise-rotations" data-toc-modified-id="block-wise-rotations-9.2"><span class="toc-item-num">9.2&nbsp;&nbsp;</span>block wise rotations</a></span></li><li><span><a href="#Rotate-matrix" data-toc-modified-id="Rotate-matrix-9.3"><span class="toc-item-num">9.3&nbsp;&nbsp;</span>Rotate matrix</a></span></li><li><span><a href="#Eigenvalue-tests" data-toc-modified-id="Eigenvalue-tests-9.4"><span class="toc-item-num">9.4&nbsp;&nbsp;</span>Eigenvalue tests</a></span></li><li><span><a href="#check-decoupling-of-blocks" data-toc-modified-id="check-decoupling-of-blocks-9.5"><span class="toc-item-num">9.5&nbsp;&nbsp;</span>check decoupling of blocks</a></span></li><li><span><a href="#check-feature-rotations" data-toc-modified-id="check-feature-rotations-9.6"><span class="toc-item-num">9.6&nbsp;&nbsp;</span>check feature rotations</a></span></li></ul></li><li><span><a href="#need-to-modify-samples-for-features-to-start-from-1" data-toc-modified-id="need-to-modify-samples-for-features-to-start-from-1-10"><span class="toc-item-num">10&nbsp;&nbsp;</span>need to modify samples for features to start from 1</a></span></li></ul></div>

In [1]:
# %load_ext autoreload
# %autoreload 2

In [1]:
import numpy as np
import torch
import json
import ase.io
from itertools import product
import matplotlib.pyplot as plt
# from rascal.representations import SphericalExpansion
import copy
from tqdm import tqdm
from ase.units import Hartree

from torch_hamiltonian_utils.torch_cg import ClebschGordanReal
from torch_hamiltonian_utils.torch_hamiltonians import fix_pyscf_l1, lowdin_orthogonalize, dense_to_blocks, blocks_to_dense, couple_blocks, decouple_blocks, hamiltonian_features
from torch_hamiltonian_utils.torch_builder import TensorBuilder

import equistore
from equistore import Labels, TensorBlock, TensorMap
from equistore_utils.librascal import  RascalSphericalExpansion, RascalPairExpansion
from equistore_utils.acdc_mini import acdc_standardize_keys, cg_increment, cg_combine
from equistore_utils.model_hamiltonian import get_feat_keys, get_feat_keys_from_uncoupled 

import importlib
torch.set_default_dtype(torch.float64)

In [2]:
def get_feat_keys_from_uncoupled(block_keys, sigma=None, order_nu=None):
    """Map UNCOUPLED block keys to corresponding feature key. take as extra input the sigma, nu value if required.
    sigma=0 returns all possible sigma values at given 'nu'"""
    blocktype, species1, n1, l1, species2, n2, l2 = block_keys
    feat_blocktype = blocktype
    keys_L=[]
    for L in range(abs(l1-l2), l1+l2+1):
        if sigma is None:
            z = (l1+l2+L)%2
            inv_sigma = 1 - 2*z
        elif abs(sigma)==1:
            inv_sigma = sigma
        else: 
            raise("Please check sigma value, it should be +1 or -1")
        
        if blocktype == 1 and n1 == n2 and l1 == l2:
            feat_blocktype = inv_sigma
                 
        if inv_sigma == -1 and blocktype == 0 and n1 == n2 and l1 == l2:
            continue     
        
        keys_L.append([(order_nu, inv_sigma, L, species1, species2, feat_blocktype)])
    #     feat= (blocktype, L,sigma,species1, species2)
    feat = Labels(["order_nu", "inversion_sigma", "spherical_harmonics_l", "species_center", "species_neighbor", "block_type"], np.asarray(keys_L, dtype=np.int32).reshape(-1,6))
    return feat

In [3]:
def lowdin_orthogonalize(fock, s):
    """
    lowdin orthogonalization of a fock matrix computing the square root of the overlap matrix
    """
    eva, eve = np.linalg.eigh(s)
    sm12 = eve @ np.diag(1.0/np.sqrt(eva)) @ eve.T
    return sm12 @ fock @ sm12

In [4]:
frames1 = ase.io.read("data/water-hamiltonian/water_coords_1000.xyz",":1")
# frames2 = ase.io.read("data/ethanol-hamiltonian/ethanol_4500.xyz", ":2")
frames = frames1 #+ frames2
for f in frames:
    f.cell = [100,100,100]
    f.positions += 50
jorbs = json.loads(json.load(open('data/water-hamiltonian/water_orbs.json', "r")))
#jorbs = json.loads(json.load(open('data/water-hamiltonian/orbs_def2_water.json', "r")))
# jorbs = json.load(open('data/ethanol-hamiltonian/orbs_saph_ethanol.json', "r"))
orbs = {}
zdic = {"O" : 8, "H":1, "C":6}
for k in jorbs:
    orbs[zdic[k]] = jorbs[k]
# 
# focks = np.load("data/water-hamiltonian/water_fock.npy", allow_pickle=True)[:len(frames1)]
# overlap = np.load("data/water-hamiltonian/water_overlap.npy", allow_pickle=True)[:len(frames1)]

# orthogonal = []
# for i in range(len(focks)): 
#     focks[i] = fix_pyscf_l1(focks[i],frames[i], orbs)
#     overlap[i] = fix_pyscf_l1(overlap[i],frames[i], orbs) 
#     orthogonal.append(lowdin_orthogonalize(focks[i], overlap[i]))
#focks = np.asarray(orthogonal, dtype=np.float64)
focks1 = np.load("data/water-hamiltonian/water_saph_orthogonal.npy", allow_pickle=True)[:len(frames1)]
# focks2 = np.load("data/ethanol-hamiltonian/ethanol_saph_orthogonal.npy", allow_pickle = True)[:len(frames2)]
focks = focks1

In [5]:
cg = ClebschGordanReal(8)

## Manipulate Hamiltonian into blocks

In [6]:
blocks = dense_to_blocks(focks, frames, orbs)
fock_bc = couple_blocks(blocks, cg)

In [7]:
fock_bc

TensorMap with 8 blocks
keys: ['block_type' 'a_i' 'n_i' 'l_i' 'a_j' 'n_j' 'l_j' 'L']
            0        8     2     0     8     2     0    0
            0        8     2     0     8     2     1    1
            0        8     2     1     8     2     1    0
            0        8     2     1     8     2     1    2
            2        1     1     0     8     2     0    0
            2        1     1     0     8     2     1    1
            0        1     1     0     1     1     0    0
            1        1     1     0     1     1     0    0

## Feature computation

In [8]:
rascal_hypers = {
    "interaction_cutoff": 4.0,
    "cutoff_smooth_width": 0.5,
    "max_radial": 6,
    "max_angular": 4,
    "gaussian_sigma_constant" : 0.2,
    "gaussian_sigma_type": "Constant",
    "compute_gradients":  False,
}

spex = RascalSphericalExpansion(rascal_hypers)
rhoi = spex.compute(frames)

lmax = rascal_hypers["max_angular"]

In [9]:
pairs = RascalPairExpansion(rascal_hypers)
gij = pairs.compute(frames)

In [10]:
rho1i = acdc_standardize_keys(rhoi)
rho1i = rho1i.keys_to_properties(['species_neighbor'])
gij =  acdc_standardize_keys(gij)

In [11]:
rho2i = cg_increment(rho1i, rho1i, lcut=lmax, other_keys_match=["species_center"], clebsch_gordan=cg)

In [12]:
#rho3i = cg_increment(rho2i, rho1i, lcut=2, other_keys_match=["species_center"], clebsch_gordan=cg)

In [13]:
rho1ij = cg_increment(rho1i, gij, lcut=lmax, other_keys_match=["species_center"], clebsch_gordan=cg)

In [14]:
#rho2ij = cg_increment(rho2i, gij, lcut=2, other_keys_match=["species_center"], clebsch_gordan=cg)

In [15]:
features = hamiltonian_features(rho2i, rho1ij)

In [16]:
from equistore.io import save
save("feature.npz", features)

## Feature Preprocessing

In [17]:
def normalize_feats(feat, all_blocks=True): 
    all_norm = 0
    normalized_blocks=[]
    for block_idx, block in feat: 
        block_norm = np.linalg.norm(block.values)
#         print(block_idx, block_norm)
        all_norm = block_norm**2 * len(block.samples) 
    
        newblock = TensorBlock(
                        values=block.values/np.sqrt(all_norm ),
                        samples=block.samples,
                        components=block.components,
                        properties= block.properties)                    
        normalized_blocks.append(newblock) 
        
    norm_feat = TensorMap(feat.keys, normalized_blocks)
    raise Exception ("Dont do it!")
    return norm_feat

In [18]:
#norm_feat = normalize_feats(features)

In [19]:
# from equistore.io import save
# save("./norm_feat.npz", norm_feat)

## Dataset

In [20]:
from equistore.io import _labels_from_npz
import equistore.operations as operations

class HamiltonianDataset(torch.utils.data.Dataset):
    #Dataset class
    def __init__(self, feature_path, target, fock, frames, feature_nu = 2):
        #
        self.features = np.load(feature_path, mmap_mode = 'r')
        #self.target = np.load(target_path, mmap_mode = 'r') 
        self.target = target #Uncoupled hamiltonian 
        #self.target = np.load(fock_path, mmap_mode = 'r')
        self.fock = torch.tensor(fock) #fock matrix
        self.keys_features = equistore.io._labels_from_npz(self.features["keys"])
        self.currentkey = self.target.keys[0]
        self.feature_nu = feature_nu
        self.frames = frames
        
        self.allfeatkey = []
        for t_key in self.target.keys:
            feature_key = self.get_feature_keys(t_key)
            self.allfeatkey.append(feature_key)
        #Remove Duplicates
        nodupes = set()
        for x in self.allfeatkey:
            if len(x) > 1:
                for z in x:
                    nodupes.add(tuple(z))
            else:
                nodupes.add(tuple(x[0]))
        
        nodupes = np.array(list(nodupes), np.int32)
        
        self.allfeatkey = Labels(names = self.allfeatkey[0].dtype.names, values = nodupes)
        

    def __len__(self):
        return len(self.frames)
    
    def __getitem__(self, structure_idx):
        feature_block, feature_key = self.generate_feature_block(self.features, structure_idx)        
        #samples_filter, target_block_samples = self.get_index_from_idx(self.target.block(self.currentkey).samples, structure_idx)

        if self.currentkey is None:
            target_block = self.fock[sorted(structure_idx)]
        else:
            target_block = operations.slice_block(self.target.block(self.currentkey), 
                                                  samples = Labels(names = ['structure'], 
                                                    values = (np.array(structure_idx)+1).reshape(-1,1)) )
        structure = [self.frames[i] for i in sorted(structure_idx)]
        #Modify feature_block to tensormap
        feature_map = TensorMap(feature_key, feature_block)
        return feature_map, target_block, structure


    def get_feature_keys(self,uncoupled_key):
        return get_feat_keys_from_uncoupled(uncoupled_key, order_nu = self.feature_nu)
    
    def generate_feature_block(self, memmap, structure_idx):
        #Generate the block from npz file
        output = []
        if self.currentkey is None:
            feature_key = self.allfeatkey
                
        else:
            feature_key = self.get_feature_keys(self.currentkey)
            
        for key in feature_key:
            block_index = list(self.keys_features).index(key)
            prefix = f"blocks/{block_index}/values"        
            block_samples = equistore.io._labels_from_npz(memmap[f"{prefix}/samples"])
            block_components = []
            for i in range(1):
                block_components.append(equistore.io._labels_from_npz(memmap[f"{prefix}/components/{i}"]))
            block_properties = equistore.io._labels_from_npz(memmap[f"{prefix}/properties"])
             

            samples_filter, block_samples = self.get_index_from_idx(block_samples, structure_idx)
            

            block_data = memmap[f"{prefix}/data"][samples_filter]
            block = TensorBlock(block_data, block_samples, block_components, block_properties)
            output.append(block)
        return output, feature_key
    
    def get_n_properties(self, memmap, key):
        block_index = list(self.keys_features).index(key)
        prefix = f"blocks/{block_index}/values"  
        block_properties = equistore.io._labels_from_npz(memmap[f"{prefix}/properties"])
        
        return len(block_properties)
    
    def get_index_from_idx(self, block_samples, structure_idx):
        #Get samples label from IDX
        samples = Labels(names = ['structure'], values = np.array(structure_idx).reshape(-1,1))
        
        all_samples = block_samples[['structure']].tolist()
        set_samples_to_slice = set(samples.tolist())
        samples_filter = np.array(
            [sample in set_samples_to_slice for sample in all_samples]
        )
        new_samples = block_samples[samples_filter]

        return samples_filter, new_samples
    
    def collate_output_values(blocks):
        feature_out = []
        target_out = []
        for sample_output in blocks:
            feature_block, target_block, structure = sample_output
            for z in feature_block:
                feature_out.append(torch.tensor(z.values))
            target_out.append(torch.tensor(target_block.values))

        return feature_out, target_out

In [21]:
feature_path = "./feature.npz"
train_dataset = HamiltonianDataset(feature_path, blocks, focks, frames)

## DataLoader

In [22]:
def collate_blocks(block_tuple):
    feature_tensor_map, target_block, structure_array = block_tuple[0]
    
    return feature_tensor_map, target_block, structure_array

In [23]:
from torch.utils.data import DataLoader, BatchSampler, SubsetRandomSampler


#Sampler = torch.utils.data.SubsetRandomSampler(range(1,len(test)+1), generator=None)
#Sampler = torch.utils.data.sampler.RandomSampler(test)
Sampler = torch.utils.data.sampler.SequentialSampler(train_dataset)
BSampler = torch.utils.data.sampler.BatchSampler(Sampler, batch_size = 50, drop_last = False)

dataloader = DataLoader(train_dataset, sampler = BSampler, collate_fn = collate_blocks)

## Model 

In [24]:
from scipy.stats import rankdata

def get_block_samples(t_key, feature_map):
    f_key = get_feat_keys_from_uncoupled(t_key, None , 2)
    ss = feature_map.block(f_key[0]).samples.copy()
    ss["structure"] = rankdata(np.abs(ss["structure"]), method='dense')
    return ss

In [25]:
class HamModel(torch.nn.Module):
    #Handles prediction of entire hamiltonian and derived results
    def __init__(self, Hamiltonian_Dataset, device, regularization=None, seed=None, layer_size=None):
        super().__init__()
#         self.features = features 
#         self.target = target
        self.models = torch.nn.ModuleDict()
        self.loss_history={}
        self.device = device
        self.target_keys = Hamiltonian_Dataset.target.keys
        self.block_samples = {}
        self.block_components = {}
        for key in Hamiltonian_Dataset.target.keys:
#             _block_type, _a_i, _n_i, _l_i, _a_j, _n_j, _l_j = key
#             target_keys = Hamiltonian_Dataset.target.keys[Hamiltonian_Dataset.target.blocks_matching(
#                 block_type = _block_type, a_i = _a_i, n_i = _n_i, l_i = _l_i, a_j = _a_j,
#                 n_j = _n_j, l_j = _l_j)]
            
            #self.block_samples[str(key)] = Hamiltonian_Dataset.target.block(key).samples
            self.block_components[str(key)] = Hamiltonian_Dataset.target.block(key).components
        
    
            n_inputs = []
            model_keys = []

            feature_keys = Hamiltonian_Dataset.get_feature_keys(key)
            for f_key in feature_keys: 
                n_features = Hamiltonian_Dataset.get_n_properties(Hamiltonian_Dataset.features, f_key)
                n_inputs.append(n_features)
                model_keys.append(f_key)
                
            n_outputs = np.ones_like(n_inputs)
                
            self.models[str(key)] = BlockModel(cg.decouple,n_inputs, n_outputs, device, model_keys, key, seed = seed, hiddenSize = layer_size)
        self.to(device)
            
    def forward(self, x):
        #Ham model uses target keys
        pred_blocks = []
        for t_key in self.target_keys:
            
            pred = self.models[str(t_key)].forward(x) #feature_tensormap must correspond to the correct features, model returns block
            
            #try:
#             print (pred.shape)
#             print ((2 * t_key['l_i'])+1)
#             print ((2 * t_key['l_j']) + 1)
            pred_block = TensorBlock(
                    values=pred.reshape((-1, (2 * t_key['l_i'])+1, (2 * t_key['l_j']) + 1, 1)), #?
                    samples = get_block_samples(t_key, x),
                    components = self.block_components[str(t_key)] ,
                    properties= Labels(["dummy"], np.asarray([[0]], dtype=np.int32))
                )
#             except:
#                 print (t_key)
#                 print (pred)
#                 print (self.block_samples[str(t_key)])
#                 print (self.block_components[str(t_key)])
                
            pred_blocks.append(pred_block)
        pred_hamiltonian = TensorMap(self.target_keys, pred_blocks)
        return(pred_hamiltonian)
    
    #write/fix forward function for train_indiv
    
    def train_individual(self, train_dataloader, regularization_dict, optimizer_type, n_epochs, loss_function, lr):
        #Iterates through the keys of self.model, then for each key we will fit self.model[key] with data[key]
        total = len(self.models)
        for index, t_key in enumerate(self.target_keys):
            print ("Now training on Block {} of {}".format(index, total))
            train_dataloader.dataset.currentkey = t_key
            
            loss_history_key = self.models[str(t_key)].fit(train_dataloader, loss_function, optimizer_type, lr, regularization_dict[str(t_key)], n_epochs)

            self.loss_history[str(t_key)] = loss_history_key
    
    def train_collective(self, train_dataloader, regularization_dict, optimizer_type, n_epochs, loss_function, lr):
        #for every loop through target keys, we predict the corresponding block and assemble the final hamiltonian
        optimizer_dict = {}
        if optimizer_type == "Adam":
            for key in train_dataloader.dataset.target.keys:
                optimizer_dict[0] = torch.optim.Adam(self.models[str(key)].parameters(), lr = lr, weight_decay = regularization_dict[str(key)])
            threshold = 200
            scheduler_threshold = 200
            tol = 0
            history_step = 1000
        
        elif optimizer_type == "LBFGS":
#             for key in train_dataloader.dataset.target.keys:
#                 optimizer_dict[str(key)] = torch.optim.LBFGS(self.models[str(key)].parameters(), lr = lr)
            optimizer_dict[0] = torch.optim.LBFGS(self.models.parameters(), lr = lr)
            threshold = 30
            scheduler_threshold = 30
            tol = 0
            history_step = 10                
        scheduler_dict = {}
        scheduler_dict[0] = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer_dict[0], factor = 0.1, patience = scheduler_threshold)
#         for key in train_dataloader.dataset.target.keys:
#             scheduler_dict[str(key)] = torch.optim.lr_scheduler.StepLR(optimizer_dict[str(key)], scheduler_threshold, gamma = 0.5)

        reg_weights = torch.tensor(list(regularization_dict.values()))
        best_state = copy.deepcopy(self.state_dict())
        lowest_loss = torch.tensor(9999)
        pred_loss = torch.tensor(0)
        trigger = 0
        loss_history = []
        pbar = tqdm(range(n_epochs))
        
        for epoch in pbar:
            pbar.set_description(f"Epoch: {epoch}")
            pbar.set_postfix(pred_loss = pred_loss.item(), lowest_loss = lowest_loss.item(), trigger = trigger)
            train_dataloader.dataset.currentkey = None
            
            for x_data, y_data, structure in train_dataloader: 
                self.collective_zg(optimizer_dict)
                #x_data, y_data = x_data.to(self.device), y_data.to(self.device)
                if optimizer_type == "LBFGS":
                    def closure():
                        self.collective_zg(optimizer_dict)
                        _pred = self.forward(x_data)
                        _pred_loss = loss_function(_pred, y_data, structure, orbs)       
                        _pred_loss = torch.nan_to_num(_pred_loss, nan=lowest_loss.item(), posinf = lowest_loss.item(), neginf = lowest_loss.item())                          
                        _reg_loss = self.get_regression_values(reg_weights) #Only works for 1 layer #Need to change!!
                        _new_loss = _pred_loss + _reg_loss
                        _new_loss.backward()
                        return _new_loss
                    for value in optimizer_dict.values():
                        value.step(closure)
#                     for param in self.parameters():
#                         print (param.grad)
                elif optimizer_type == "Adam":
                    pred = self.forward(x_data)
                    pred_loss = loss_function(pred, y_data, structure, orbs)  
#                     reg_loss = torch.sum(torch.pow(self.nn.weight,2))#Only works for 1 layer
                    new_loss = pred_loss 
                    new_loss.backward()
                    self.collective_step(optimizer_dict)
            with torch.no_grad():
                current_loss = 0 
                for x_data, y_data, structure in train_dataloader:
                    pred = self.forward(x_data)
                    current_loss  += loss_function(pred, y_data, structure, orbs)   #Loss should be normalized already
                pred_loss = current_loss
                reg_loss = self.get_regression_values(reg_weights)#Only works for 1 layer
                new_loss = pred_loss + reg_loss
                for scheduler in scheduler_dict.values():
                    scheduler.step(new_loss)
                if pred_loss >100000 or (pred_loss.isnan().any()) :
                    print ("Optimizer shows weird behaviour, reinitializing at previous best_State")
                    self.load_state_dict(best_state)
                    if optimizer_type == "Adam":
                        optimizer = torch.optim.Adam(self.parameters(), lr = lr, weight_decay = regularization_dict[str(key)])
                    elif optimizer_type == "LBFGS":
                        optimizer = torch.optim.LBFGS(self.parameters(), lr = lr)

                if epoch % history_step == 1:
                    loss_history.append(lowest_loss.item())
                
                if lowest_loss - new_loss > tol: #threshold to stop training             
                    best_state = copy.deepcopy(self.state_dict())
                    lowest_loss = new_loss 
                    trigger = 0 
                    
                    
                else:
                    trigger += 1
                    if trigger > threshold:
                        self.load_state_dict(best_state)
                        print ("Implemented early stopping with lowest_loss: {}".format(lowest_loss))
                        return loss_history
        return loss_history
        
    def collective_step(self, dictionary):
        for value in dictionary.values():
            value.step()
            
    def collective_zg(self, dictionary):
        for value in dictionary.values():
            value.zero_grad()
    
    def get_regression_values(self, reg_weights):
        output = []
        for param in self.parameters():
            output.append(torch.sum(torch.pow(param,2)))
        try:
            output = torch.sum(torch.tensor(output) * reg_weights)
        except:
            output = 0
        return output

In [26]:
class BlockModel(torch.nn.Module): #Currently only 1 model per block
    def __init__(self, reconstruction_function, inputSize, outputSize, device, keys, target_key, seed = None, hiddenSize = None):
        super().__init__()
        self.reconstruction_function = reconstruction_function
        self.inputSize = inputSize
        self.outputSize = outputSize
        self.device = device
        self.keys = keys
        self.target_key = target_key
        self.hiddenSize = hiddenSize
        self.initialize_model(seed)
        
        self.to(device)
    
    def initialize_model(self, seed):
        
        if seed is not None:
            torch.manual_seed(seed)
        
        self.models = torch.nn.ModuleDict()
        for index, key in enumerate(self.keys):
            if key['spherical_harmonics_l'] == 0:
#                 self.models[str(key)] = torch.nn.Linear(self.inputSize[index], self.outputSize[index], bias = False)

                self.models[str(key)] = InvariantNonLinearModel(self.inputSize[index], self.hiddenSize, self.outputSize[index])
            else: 
                self.models[str(key)] = torch.nn.Linear(self.inputSize[index], self.outputSize[index], bias = False)
        
    def forward(self, feature_tensormap):
        #Block model uses feature keys
        pred_values = {}
        for key in self.keys:
            #print(key)
            feature_values = feature_tensormap.block(key).values
            d1, d2, d3 = feature_values.shape
            L = int((d2 -1)/2)
            
            pred = self.models[str(key)](torch.tensor(feature_values.reshape(d1 * d2, d3)))
            pred = pred.reshape(d1,d2)
            pred_values[L] = pred
        pred_block_values = self.reconstruction_function({(self.target_key['l_i'],self.target_key['l_j']) : pred_values})
        
        #DOES NOT WORK FOR BATCHES

        #pred = torch.hstack(pred_values)
        #pred = self.reconstruction_function(pred_values)
        return pred_block_values 
        #return pred_values

    
    def fit(self,traindata_loader, loss_function, optimizer_type, lr, reg, n_epochs):
        if optimizer_type == "Adam":
            optimizer = torch.optim.Adam(self.parameters(), lr = lr, weight_decay = reg.item())
            threshold = 200
            scheduler_threshold = 50
            tol = 0
            history_step = 1000
        
        elif optimizer_type == "LBFGS":
            optimizer = torch.optim.LBFGS(self.parameters(), lr = lr)
            threshold = 30
            scheduler_threshold = 10
            tol = 0
            history_step = 10
            
        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor = 0.1, patience = scheduler_threshold)      
        best_state = copy.deepcopy(self.state_dict())
        lowest_loss = torch.tensor(9999)
        pred_loss = torch.tensor(0)
        trigger = 0
        loss_history = []
        pbar = tqdm(range(n_epochs))
        
        for epoch in pbar:
            pbar.set_description(f"Epoch: {epoch}")
            pbar.set_postfix(pred_loss = pred_loss.item(), lowest_loss = lowest_loss.item(), trigger = trigger)
            
            for x_data, y_data, structure in traindata_loader: 
                optimizer.zero_grad()
                #x_data, y_data = x_data.to(self.device), y_data.to(self.device)
                if optimizer_type == "LBFGS":
                    def closure():
                        optimizer.zero_grad()
                        _pred = self.forward(x_data)                                        
                        _pred_loss = loss_function(_pred, y_data.values)
                        _pred_loss = torch.nan_to_num(_pred_loss, nan=lowest_loss.item(), posinf = lowest_loss.item(), neginf = lowest_loss.item())                 
                        _reg_loss = self.get_regression_values(reg.item()) #Only works for 1 layer
                        _new_loss = _pred_loss + _reg_loss
                        _new_loss.backward()
                        return _new_loss
                    optimizer.step(closure)

                elif optimizer_type == "Adam":
                    pred = self.forward(x_data)
                    pred_loss = loss_function(pred, y_data.values)
                    #reg_loss = self.get_regression_values(reg.item())#Only works for 1 layer
                    new_loss = pred_loss #+ reg_loss
                    new_loss.backward()

                    optimizer.step()
                
            with torch.no_grad():
                current_loss = 0 
                for x_data, y_data, structure in traindata_loader:
                    pred = self.forward(x_data)
                    current_loss  += loss_function(pred, y_data.values) #Loss should be normalized already
                pred_loss = current_loss
                reg_loss = self.get_regression_values(reg.item()) 
                new_loss = pred_loss + reg_loss
                scheduler.step(new_loss)
                if pred_loss >100000 or (pred_loss.isnan().any()) :
                    print ("Optimizer shows weird behaviour, reinitializing at previous best_State")
                    self.load_state_dict(best_state)
                    if optimizer_type == "Adam":
                        optimizer = torch.optim.Adam(self.parameters(), lr = lr, weight_decay = reg.item())
                    elif optimizer_type == "LBFGS":
                        optimizer = torch.optim.LBFGS(self.parameters(), lr = lr)

                if epoch % history_step == 1:
                    loss_history.append(lowest_loss.item())
                
                if lowest_loss - new_loss > tol: #threshold to stop training
                    best_state = copy.deepcopy(self.state_dict())
                    lowest_loss = new_loss 
                    trigger = 0 
                    
                    
                else:
                    trigger += 1
                    if trigger > threshold:
                        self.load_state_dict(best_state)
                        print ("Implemented early stopping with lowest_loss: {}".format(lowest_loss))
                        return loss_history
        return loss_history
    
    def get_regression_values(self, reg_weights):
        output = []
        for param in self.parameters():
            output.append(torch.sum(torch.pow(param,2)))
        try:
            output = torch.sum(torch.tensor(output) * reg_weights)
        except:
            output = 0
        return output

In [27]:
# class NonLinearModel(torch.nn.Module):
#     def __init__(
#         self,
#         input_size: int,
#         hidden_size: int,
#         last_hidden_size: int,
#         output_size: int,
#     ):
#         """ """
#         super().__init__()
#         self.input_layer = torch.nn.Linear(input_size, last_hidden_size)
        
#         self.invariant_nn = torch.nn.Sequential(
#             torch.nn.Linear(input_size, hidden_size),
#             torch.nn.Tanh(),
#             torch.nn.Linear(hidden_size, hidden_size),
#             torch.nn.Tanh(),
#             torch.nn.Linear(hidden_size, last_hidden_size),
#         )
#         # Define the output layer that makes the prediction
#         self.output_layer = torch.nn.Linear(last_hidden_size, output_size)

#     def forward(self, input: torch.Tensor, invariant: torch.Tensor):
#         # H-stack the invariant along the components dimension so that there are
#         # (2 \lambda + 1) copies and the dimensions match the equivariant
#         invariant = torch.hstack([invariant] * input.values.shape[1])

#         # Pass the invariant tensor through the NN to create a nonlinear
#         # multiplier. Also pass the equivariant through a linear input layer.
#         nonlinear_multiplier = self.invariant_nn(invariant)
#         linear_input = self.input_layer(input)

#         # Perform element-wise (Hadamard) multiplication of the transformed
#         # input with the nonlinear multiplier, which now have the same dimensions
#         nonlinear_input = torch.mul(linear_input, nonlinear_multiplier)

#         # Finally pass through the output layer and return
#         return self.output_layer(nonlinear_input)

In [38]:
class InvariantNonLinearModel(torch.nn.Module):
    def __init__(
        self,
        input_size: int,
        hidden_size: int,
        output_size: int,
    ):
        """ """
        super().__init__()
                
        self.invariant_nn = torch.nn.Sequential(
            torch.nn.Linear(input_size, hidden_size),
            torch.nn.Softmax(),
            #torch.nn.Linear(hidden_size, hidden_size),
            #torch.nn.Softmax(),
            #torch.nn.Linear(hidden_size, hidden_size),
            #torch.nn.Tanh(),
            torch.nn.Linear(hidden_size, output_size)
        )
        # Define the output layer that makes the prediction
        #self.output_layer = torch.nn.Linear(last_hidden_size, output_size)

    def forward(self, input: torch.Tensor):
        result = self.invariant_nn(input)
        
        return result

## Training

In [34]:
def mse_block_values(pred, true):
    true = true.reshape(true.shape[:-1]) 
    MSE = torch.sum(torch.pow(true - pred,2)) / torch.numel(true)
    return MSE*10**7

def mse_full(pred_blocks, fock,frame, orbs):
    predicted = blocks_to_dense(pred_blocks, frame, orbs)
    #fock = torch.tensor(focks)
    #print (mse_full_blockwise(pred_blocks, blocks, frame, orbs))
    mse_loss = torch.empty(len(frame))
    for i in range(len(frame)):
        mse_loss[i] = ((torch.linalg.norm(fock[i]-predicted[i]))**2)/torch.numel(fock[i])
        #print("from mse", i, fock[i], mse_loss[i])
    return torch.mean(mse_loss)*(Hartree)**2#, mse_loss

def mse_full_blockwise(pred_blocks, block_tensormap, frame, orbs):
    indiv_mse = torch.zeros(1)
    for key,block in pred_blocks:
        #MSE = ((torch.linalg.norm(block_tensormap.block(key).values-block.values))**2)
        print (key)
        print (((torch.linalg.norm(block_tensormap.block(key).values- block.values))**2)/ torch.numel(block_tensormap.block(key).values))
        print (torch.sum(torch.pow(block_tensormap.block(key).values - block.values, 2)) / torch.numel(block_tensormap.block(key).values))
        MSE = torch.sum(torch.pow(block_tensormap.block(key).values - block.values, 2)) / torch.numel(block_tensormap.block(key).values)
        indiv_mse += MSE
    
    return (indiv_mse/len(frame))*(Hartree)**2, indiv_mse
    
def mse_eigvals(pred_blocks, fock, frame, orbs):
    fock = torch.tensor(focks)
    predicted = blocks_to_dense(pred_blocks, frame, orbs)
    evanorm = torch.empty(len(frame))
    for i in range(len(frame)):
        evanorm[i] = torch.mean((torch.linalg.eigvalsh(fock[i]) - torch.linalg.eigvalsh(predicted[i]))**2)/len(fock[i])
    return torch.mean(evanorm)*(Hartree)**2

In [42]:
model = HamModel(train_dataset, "cpu", None, None, 128)

In [43]:
optimizer = torch.optim.LBFGS(
        model.parameters(),
        lr=1,  line_search_fn="strong_wolfe",
        history_size=256, tolerance_grad=1e-12, tolerance_change=1e-12
    )

In [44]:
#doesn't go to zero even for a single structure
#problem with the model probably
all_losses = []
for epoch in range(300):
    dataloader.dataset.currentkey = None
    for x_data, y_data, structure in dataloader:    
        def single_step():
            optimizer.zero_grad()
            pred = model(x_data)
            loss = mse_full(pred, y_data, structure, orbs)
            loss.backward()
            return loss

        loss = optimizer.step(single_step)

        all_losses.append(loss.item())

        if epoch % 1 == 0:
            print(epoch, loss.item()) 

0 55.5356490575776
1 2.2535158907136936
2 1.9193001349494911
3 1.1616664058699668
4 0.7913797397891894
5 0.7776851541994158
6 0.7637541226173614
7 0.7425411385161687
8 0.6916919411258343
9 0.628322635896238
10 0.5202032879056664
11 0.42138410016019495
12 0.3519357216236843
13 0.3180832763038013
14 0.31802370550769854
15 0.3177493243829828
16 0.3175384599524493
17 0.3170098493519389
18 0.3160637574946855
19 0.3150284089473091
20 0.31413487075004054
21 0.31182756506280335
22 0.3111409548983539
23 0.3102014282052689
24 0.3081065934813937
25 0.3073852822820888
26 0.3068535985152003
27 0.3055172578309105
28 0.3036760690984036
29 0.3028633443125968
30 0.30246516731134043
31 0.3017674525434194
32 0.3002563203483206
33 0.2971525960387195
34 0.2937440038294316
35 0.29053717042166954
36 0.286113593118678
37 0.2818007871302119
38 0.278130040955328
39 0.27364210065105843
40 0.26928656583657434
41 0.2652128418047148
42 0.2609029454593501
43 0.25682269429345783
44 0.2530241625955775
45 0.24845105803

KeyboardInterrupt: 

In [38]:
all_losses = []
for epoch in range(5000):
    dataloader.dataset.currentkey = None
    for x_data, y_data, structure in dataloader:
        optimizer.zero_grad()
        pred = model(x_data)
        loss = mse_full(pred, y_data, structure, orbs)
        loss.backward()
        optimizer.step()
        #scheduler.step()

        all_losses.append(loss.item())

        if epoch % 50 == 0:
            print(epoch, loss.item()) 

TypeError: step() missing 1 required positional argument: 'closure'

## Evaluation

In [55]:
#Load test set
test_frames1 = ase.io.read("data/water-hamiltonian/water_coords_1000.xyz","1:2")
# frames2 = ase.io.read("data/ethanol-hamiltonian/ethanol_4500.xyz", ":2")
test_frames = test_frames1 #+ frames2
for f in test_frames:
    f.cell = [100,100,100]
    f.positions += 50

# test_focks1 = np.load("data/water-hamiltonian/water_saph_orthogonal.npy", allow_pickle=True)[50:60]
# focks2 = np.load("data/ethanol-hamiltonian/ethanol_saph_orthogonal.npy", allow_pickle = True)[:len(frames2)]

test_focks1 = np.load("data/water-hamiltonian/water_saph_orthogonal.npy", allow_pickle=True)[50:80]
test_focks = test_focks1
# test_focks = np.load("data/water-hamiltonian/water_fock.npy", allow_pickle=True)[50:80]
# test_overlap = np.load("data/water-hamiltonian/water_overlap.npy", allow_pickle=True)[50:80]

# test_orthogonal = []
# for i in range(len(test_focks)): 
#     test_focks[i] = fix_pyscf_l1(test_focks[i],test_frames[i], orbs)
#     test_overlap[i] = fix_pyscf_l1(test_overlap[i],test_frames[i], orbs)
#     test_orthogonal.append(lowdin_orthogonalize(test_focks[i], test_overlap[i]))
# test_focks = np.asarray(test_orthogonal, dtype=np.float64)
    
test_blocks = dense_to_blocks(test_focks, test_frames, orbs)
test_fock_bc = couple_blocks(test_blocks, cg)

In [56]:
test_rhoi = spex.compute(test_frames)
test_gij = pairs.compute(test_frames)
test_rho1i = acdc_standardize_keys(test_rhoi)
test_rho1i = test_rho1i.keys_to_properties(['species_neighbor'])
test_gij =  acdc_standardize_keys(test_gij)
test_rho2i = cg_increment(test_rho1i, test_rho1i, lcut=lmax, other_keys_match=["species_center"], clebsch_gordan=cg)
test_rho1ij = cg_increment(test_rho1i, test_gij, lcut=lmax, other_keys_match=["species_center"], clebsch_gordan=cg)

test_features = hamiltonian_features(test_rho2i, test_rho1ij)

In [57]:
from equistore.io import save
save("test_feature.npz", test_features)

In [58]:
# norm_test_feat = normalize_feats(test_features)
# from equistore.io import save
# save("./norm_testfeat.npz", norm_test_feat)

In [59]:
#test_target_path = "./test_fock.npz"
test_feature_path = "./test_feature.npz"
testing = HamiltonianDataset(test_feature_path, test_blocks, test_focks, test_frames)

In [60]:
from torch.utils.data import DataLoader, BatchSampler, SubsetRandomSampler


#Sampler = torch.utils.data.SubsetRandomSampler(range(1,len(test)+1), generator=None)
test_Sampler = torch.utils.data.sampler.RandomSampler(testing)
test_BSampler = torch.utils.data.sampler.BatchSampler(test_Sampler, batch_size = 50, drop_last = False)

test_dataloader = DataLoader(testing, sampler = test_BSampler, collate_fn = collate_blocks)

In [393]:
def mse_block_values(pred, true):
    true = true.reshape(true.shape[:-1])
    MSE = torch.sum(torch.pow(true - pred,2)) / torch.numel(true)
    return MSE

def mse_full(pred_blocks, fock,frame, orbs):
    predicted = blocks_to_dense(pred_blocks, frame, orbs)
    #fock = torch.tensor(focks)
    mse_loss = torch.empty(len(frame))
    for i in range(len(frame)):
        mse_loss[i] = ((torch.linalg.norm(fock[i]-predicted[i]))**2)#/torch.numel(fock[i])

        #print("from mse", i, fock[i], mse_loss[i])
    return torch.mean(mse_loss)*(Hartree)**2

def mse_eigvals(pred_blocks, fock, frame, orbs):
    #fock = torch.tensor(focks)
    predicted = blocks_to_dense(pred_blocks, frame, orbs)
    evanorm = torch.empty(len(frame))
    for i in range(len(frame)):
        evanorm[i] = torch.mean((torch.linalg.eigvalsh(fock[i]) - torch.linalg.eigvalsh(predicted[i]))**2)/len(fock[i])
    return torch.mean(evanorm)*(Hartree)**2

In [61]:
testing.currentkey = None
dataloader.dataset.currentkey = None
for x_data, y_data, structures in test_dataloader:
    t_pred = model(x_data)
    print (mse_full(t_pred, torch.tensor(test_focks), structures, orbs))

tensor(0.5713, grad_fn=<MulBackward0>)
